In [23]:
import json
import spacy
import pandas as pd
import numpy as np

nlp = spacy.load('en_core_web_sm')

depth = 1
proofwriter_train = pd.read_json(f'proofwriter-dataset-V2020.12.3/OWA/depth-{depth}/meta-train.jsonl', lines=True)
proofwriter_test = pd.read_json(f'proofwriter-dataset-V2020.12.3/OWA/depth-{depth}/meta-test.jsonl', lines=True)

def get_dataset(df: pd.DataFrame):
    examples = []
    for index, row in df.iterrows():
        premises = row['theory']
        premises = premises.replace('. ', '\n ')
        question_dict = row['questions']
        num_q = len(question_dict)
        questions, answers = [], []
        for i in range(1, num_q + 1):
            questions.append(question_dict[f'Q{i}']['question'])
            answers.append(question_dict[f'Q{i}']['answer'])
        examples.append(dict(
            premises=premises,
            conclusions=questions,
            labels=answers
        ))
    return examples

train_examples = get_dataset(proofwriter_train)
test_examples = get_dataset(proofwriter_test)

In [15]:
glove_dict = dict()
with open('glove.6B/glove.6B.50d.txt', 'r') as fp:
    glove = fp.read().splitlines()
for line in glove:
    line =  line.split(' ')
    word, nums = line[0], np.array([float(num) for num in line[1:]])
    glove_dict[word] = nums

In [18]:
for idx, example in enumerate(train_examples):
   ...

train_examples[0]['premises']

p_vec = sentence2vec(premise)
for q in train_examples[0]['questions']:
   ... 
q_vec_list = []

'Anne is not blue\n Anne is cold\n Anne is round\n Fiona is blue\n Fiona is furry\n Gary is furry\n Gary is quiet\n Gary is smart\n Harry is cold\n Harry is quiet\n If something is furry and not blue then it is nice\n If Anne is furry then Anne is nice\n Smart, furry things are round.'

In [44]:
def sentence2vec(sentence: str):
    doc = nlp(sentence)
    tokens = [word.lower_ for word in doc]
    print(tokens)
    vecs = np.stack([glove_dict[tok] for tok in tokens if tok in glove_dict])
    return np.mean(vecs, axis=0)

In [45]:
sentence2vec(train_examples[0]['premises'])

['anne', 'is', 'not', 'blue', '\n ', 'anne', 'is', 'cold', '\n ', 'anne', 'is', 'round', '\n ', 'fiona', 'is', 'blue', '\n ', 'fiona', 'is', 'furry', '\n ', 'gary', 'is', 'furry', '\n ', 'gary', 'is', 'quiet', '\n ', 'gary', 'is', 'smart', '\n ', 'harry', 'is', 'cold', '\n ', 'harry', 'is', 'quiet', '\n ', 'if', 'something', 'is', 'furry', 'and', 'not', 'blue', 'then', 'it', 'is', 'nice', '\n ', 'if', 'anne', 'is', 'furry', 'then', 'anne', 'is', 'nice', '\n ', 'smart', ',', 'furry', 'things', 'are', 'round', '.']
(57, 50)


array([ 2.12484053e-01,  3.65212593e-01, -3.74428477e-01,  7.13554386e-03,
        5.23062456e-01,  3.15265491e-01, -4.09569112e-01, -2.21966772e-01,
       -4.77918421e-02, -1.85236772e-01, -4.10184316e-02,  3.45472596e-01,
       -2.12867649e-01, -1.14557316e-01,  3.21596246e-01,  2.88775860e-01,
        2.59213035e-01,  1.15913351e-01, -2.96744772e-01, -3.91666246e-01,
       -3.29968053e-01,  2.34274961e-01,  1.89931509e-01,  2.03444561e-01,
        1.77874070e-01, -1.28888088e+00, -3.74467123e-01,  3.97256509e-01,
        3.06357018e-01, -2.04219719e-01,  2.49035149e+00, -9.37287649e-02,
       -9.29462982e-02, -1.65949877e-01,  2.35537719e-02, -8.09517193e-02,
       -9.25543860e-04,  1.49141179e-01,  6.44282895e-02, -4.39663491e-01,
        9.01704968e-02,  2.69646158e-01, -1.19467211e-01, -8.26754386e-03,
        6.75170263e-02,  1.94488249e-01, -5.32562281e-02, -5.65375263e-01,
       -4.37101930e-02,  4.30473193e-01])

In [ ]:
'anne' in glove_dict